In [2]:
import pandas as pd
import json 
import os 
import hana_ml
from hana_ml import ConnectionContext

print(pd.__version__)
print(hana_ml.__version__)

/var/folders/3v/kt3p4yx106n_9h7p5_4lqk7c0000gn/T/ipykernel_45534/832702571.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


2.2.0
2.19.24022101


### Connect HANA Table

In [2]:
# Connect HANA Table 
with open(os.path.join(os.getcwd(), '/Users/i551982/Desktop/Github/enablement-session-01/HANA-configure-chul.json')) as f:
    hana_env_c = json.load(f)
    port_c  = hana_env_c['port']
    user_c  = hana_env_c['user']
    url_c  = hana_env_c['url']
    pwd_c  = hana_env_c['pwd']

cc = ConnectionContext(address=url_c, port=port_c, user= user_c, password = pwd_c, encrypt=True)
cursor = cc.connection.cursor()
cursor.execute("""SET SCHEMA GEN_AI""")

#Check hana_ml version 
print(cc.hana_version())
#Check current schema 
print(cc.get_current_schema())


4.00.000.00.1707826119 (CE2024.4)
GEN_AI


In [3]:
print(cc.get_tables())

        TABLE_NAME
0     DEMO_BLOCK_1
1     DEMO_BLOCK_2
2     DEMO_BLOCK_3
3  DEMO_BLOCK_TEST
4  TEST_DEMO_BLOCK
5      VECTOR_TEST


In [4]:
data = pd.read_csv('/Users/i551982/Desktop/Github/enablement-session-01/data/block_sample_kr_v3.1.csv', index_col=False)
# print(data.columns)

data['BlockID'] = range(1, len(data) + 1)
print(data.columns)


Index(['MaterialNumber', 'MaterialName', 'PurchasingGroupDescription',
       'PurchasingGroup', 'MaterialDescription', 'Total_stock',
       'Unrestricted_use_stock', 'Blocked_stock', 'Block_reason',
       'Blocked_date', 'Solution_date', 'Solution',
       'PurchasingGroupDescription_KR', 'MaterialDescription_KR',
       'Solution_KR', 'Block_reason_KR', 'BlockID'],
      dtype='object')


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8020 entries, 0 to 8019
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   MaterialNumber                 8020 non-null   int64 
 1   MaterialName                   8020 non-null   object
 2   PurchasingGroupDescription     8020 non-null   object
 3   PurchasingGroup                8020 non-null   object
 4   MaterialDescription            8020 non-null   object
 5   Total_stock                    8020 non-null   int64 
 6   Unrestricted_use_stock         8020 non-null   int64 
 7   Blocked_stock                  8020 non-null   int64 
 8   Block_reason                   8020 non-null   object
 9   Blocked_date                   8020 non-null   object
 10  Solution_date                  8020 non-null   object
 11  Solution                       8020 non-null   object
 12  PurchasingGroupDescription_KR  8020 non-null   object
 13  Mat

### Column re-arrange

In [6]:
data1 = data[['BlockID','MaterialNumber', 'MaterialName', 'MaterialDescription', 'MaterialDescription_KR',
              'PurchasingGroup','PurchasingGroupDescription','PurchasingGroupDescription_KR',
              'Total_stock','Unrestricted_use_stock', 'Blocked_stock',
              'Blocked_date', 'Block_reason','Block_reason_KR',
              'Solution', 'Solution_KR', 'Solution_date' ]]

In [7]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8020 entries, 0 to 8019
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   BlockID                        8020 non-null   int64 
 1   MaterialNumber                 8020 non-null   int64 
 2   MaterialName                   8020 non-null   object
 3   MaterialDescription            8020 non-null   object
 4   MaterialDescription_KR         7996 non-null   object
 5   PurchasingGroup                8020 non-null   object
 6   PurchasingGroupDescription     8020 non-null   object
 7   PurchasingGroupDescription_KR  8020 non-null   object
 8   Total_stock                    8020 non-null   int64 
 9   Unrestricted_use_stock         8020 non-null   int64 
 10  Blocked_stock                  8020 non-null   int64 
 11  Blocked_date                   8020 non-null   object
 12  Block_reason                   8020 non-null   object
 13  Blo

In [8]:
data1.head()

,BlockID,MaterialNumber,MaterialName,MaterialDescription,MaterialDescription_KR,PurchasingGroup,PurchasingGroupDescription,PurchasingGroupDescription_KR,Total_stock,Unrestricted_use_stock,Blocked_stock,Blocked_date,Block_reason,Block_reason_KR,Solution,Solution_KR,Solution_date
0,1,368814,NORDVIKEN,"Bar table, 140x80 cm","바 테이블, 140x80 cm",P01,Bar furniture,바 가구,48663,107,48556,3/1/24,Identification of packaging defects that compr...,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 시정 조치가 필요...,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",4/28/24
1,2,368814,NORDVIKEN,"Bar table, 140x80 cm","바 테이블, 140x80 cm",P01,Bar furniture,바 가구,48663,107,48556,3/1/24,Identification of packaging defects that compr...,출하 전에 제품의 무결성을 손상시키는 포장 결함을 식별하여 즉각적인 수정 조치가 필...,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",4/28/24
2,3,368814,NORDVIKEN,"Bar table, 140x80 cm","바 테이블, 140x80 cm",P01,Bar furniture,바 가구,48663,107,48556,3/1/24,Identification of packaging defects that compr...,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 수정 조치가 필요...,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",4/28/24
3,4,368814,NORDVIKEN,"Bar table, 140x80 cm","바 테이블, 140x80 cm",P01,Bar furniture,바 가구,48663,107,48556,3/1/24,Identification of packaging defects that compr...,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 수정 조치가 필요...,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",4/28/24
4,5,368814,NORDVIKEN,"Bar table, 140x80 cm","바 테이블, 140x80 cm",P01,Bar furniture,바 가구,48663,107,48556,3/1/24,Identification of packaging defects that compr...,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별함으로써 즉각적인 수정 조치가 ...,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",4/28/24


### Create Table
Table Name: DEMO-BLOCK3

In [26]:
cursor = cc.connection.cursor()
sql_command = '''CREATE TABLE "DEMO_BLOCK_3" ( 
    "BlockID" BIGINT, 
    "MaterialNumber" BIGINT,
    "MaterialName" NCLOB MEMORY THRESHOLD 0,
    "MaterialDescription" NCLOB MEMORY THRESHOLD 0,
    "MaterialDescription_KR" NCLOB MEMORY THRESHOLD 0,
    "PurchasingGroup"  NVARCHAR(100), 
    "PurchasingGroupDescription" NVARCHAR(5000), 
    "PurchasingGroupDescription_KR" NVARCHAR(5000), 
    "Total_stock" BIGINT,
    "Unrestricted_use_stock" BIGINT, 
    "Blocked_stock" BIGINT, 
    "Blocked_date" DATE,
    "Block_reason" NCLOB MEMORY THRESHOLD 0,
    "Block_reason_KR" NCLOB MEMORY THRESHOLD 0,
    "Solution" NCLOB MEMORY THRESHOLD 0,
    "Solution_KR" NCLOB MEMORY THRESHOLD 0,
    "Solution_date" DATE)'''
    
cursor.execute(sql_command)
cursor.close()

### Read table

In [27]:
cc.has_schema('GEN_AI')
cc.get_tables()

,TABLE_NAME
0,DEMO_BLOCK_1
1,DEMO_BLOCK_2
2,DEMO_BLOCK_3
3,DEMO_BLOCK_TEST
4,TEST_DEMO_BLOCK
5,VECTOR_TEST


In [28]:
cc.sql('''SELECT TOP 10 *  FROM DEMO_BLOCK_3''').collect().head()

,BlockID,MaterialNumber,MaterialName,MaterialDescription,MaterialDescription_KR,PurchasingGroup,PurchasingGroupDescription,PurchasingGroupDescription_KR,Total_stock,Unrestricted_use_stock,Blocked_stock,Blocked_date,Block_reason,Block_reason_KR,Solution,Solution_KR,Solution_date


### Load dataset into HANA

In [29]:
data1.columns

Index(['BlockID', 'MaterialNumber', 'MaterialName', 'MaterialDescription',
       'MaterialDescription_KR', 'PurchasingGroup',
       'PurchasingGroupDescription', 'PurchasingGroupDescription_KR',
       'Total_stock', 'Unrestricted_use_stock', 'Blocked_stock',
       'Blocked_date', 'Block_reason', 'Block_reason_KR', 'Solution',
       'Solution_KR', 'Solution_date'],
      dtype='object')

In [30]:
data2 = data1.copy()
data2['Blocked_date'] = pd.to_datetime(data2['Blocked_date'])
data2['Solution_date'] = pd.to_datetime(data2['Solution_date'])

/var/folders/3v/kt3p4yx106n_9h7p5_4lqk7c0000gn/T/ipykernel_3529/3690285358.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data2['Blocked_date'] = pd.to_datetime(data2['Blocked_date'])
/var/folders/3v/kt3p4yx106n_9h7p5_4lqk7c0000gn/T/ipykernel_3529/3690285358.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data2['Solution_date'] = pd.to_datetime(data2['Solution_date'])


In [33]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8020 entries, 0 to 8019
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   BlockID                        8020 non-null   int64         
 1   MaterialNumber                 8020 non-null   int64         
 2   MaterialName                   8020 non-null   object        
 3   MaterialDescription            8020 non-null   object        
 4   MaterialDescription_KR         7996 non-null   object        
 5   PurchasingGroup                8020 non-null   object        
 6   PurchasingGroupDescription     8020 non-null   object        
 7   PurchasingGroupDescription_KR  8020 non-null   object        
 8   Total_stock                    8020 non-null   int64         
 9   Unrestricted_use_stock         8020 non-null   int64         
 10  Blocked_stock                  8020 non-null   int64         
 11  Blocked_date     

In [36]:
data2.shape

(8020, 17)

In [37]:
# import dataframe into hana table 
from hana_ml.dataframe import create_dataframe_from_pandas
v_hdf = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=data2,
    table_name="DEMO_BLOCK_3", 
    allow_bigint=True, 
    append=True,
    force=False)

100%|██████████| 1/1 [00:19<00:00, 19.61s/it]


In [38]:
# Understand the data in this newly created table
# you could join multiple text columns in SQL
hdf = cc.sql('''SELECT TOP 10 "BlockID","MaterialNumber", "MaterialName",
             "Blocked_stock","Block_reason","Blocked_date", "Solution", "Solution_KR","Solution_date" 
             FROM DEMO_BLOCK_3''')
df_abstract = hdf.collect()
df_abstract.head()

,BlockID,MaterialNumber,MaterialName,Blocked_stock,Block_reason,Blocked_date,Solution,Solution_KR,Solution_date
0,1,368814,NORDVIKEN,48556,Identification of packaging defects that compr...,2024-03-01,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28
1,2,368814,NORDVIKEN,48556,Identification of packaging defects that compr...,2024-03-01,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28
2,3,368814,NORDVIKEN,48556,Identification of packaging defects that compr...,2024-03-01,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28
3,4,368814,NORDVIKEN,48556,Identification of packaging defects that compr...,2024-03-01,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28
4,5,368814,NORDVIKEN,48556,Identification of packaging defects that compr...,2024-03-01,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28


### Cteate Context column

In [49]:
data2.columns

Index(['BlockID', 'MaterialNumber', 'MaterialName', 'MaterialDescription',
       'MaterialDescription_KR', 'PurchasingGroup',
       'PurchasingGroupDescription', 'PurchasingGroupDescription_KR',
       'Total_stock', 'Unrestricted_use_stock', 'Blocked_stock',
       'Blocked_date', 'Block_reason', 'Block_reason_KR', 'Solution',
       'Solution_KR', 'Solution_date', 'Context', 'Context_KR'],
      dtype='object')

In [57]:
data2['Context'] = 'Block reason:' + data2['Block_reason'] \
                    + '\n Material Number:' + data2['MaterialNumber'].astype(str) \
                    + '\n Material Name:' + data2['MaterialName'] \
                    + '\n Material Description:' + data2['MaterialDescription'] \
                    + '\n Purchasing Group:' + data2['PurchasingGroup'] \
                    + '\n Purchasing Group Description:' + data2['PurchasingGroupDescription']
                    
data2['Context_KR'] = '블럭 원인:' + data2['Block_reason_KR'] \
                    + '\n 자재 번호:' + data2['MaterialNumber'].astype(str) \
                    + '\n 자재명:' + data2['MaterialName'] \
                    + '\n 자재 설명:' + data2['MaterialDescription_KR'] \
                    + '\n 자재 그룹:' + data2['PurchasingGroup'] \
                    + '\n 구매 그룹 설명:' + data2['PurchasingGroupDescription_KR']

In [51]:
data2.head()

,BlockID,MaterialNumber,MaterialName,MaterialDescription,MaterialDescription_KR,PurchasingGroup,PurchasingGroupDescription,PurchasingGroupDescription_KR,Total_stock,Unrestricted_use_stock,Blocked_stock,Blocked_date,Block_reason,Block_reason_KR,Solution,Solution_KR,Solution_date,Context,Context_KR
0,1,368814,NORDVIKEN,"Bar table, 140x80 cm","바 테이블, 140x80 cm",P01,Bar furniture,바 가구,48663,107,48556,2024-03-01,Identification of packaging defects that compr...,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 시정 조치가 필요...,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28,Block reason:Identification of packaging defec...,블럭 원인:출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 시정 ...
1,2,368814,NORDVIKEN,"Bar table, 140x80 cm","바 테이블, 140x80 cm",P01,Bar furniture,바 가구,48663,107,48556,2024-03-01,Identification of packaging defects that compr...,출하 전에 제품의 무결성을 손상시키는 포장 결함을 식별하여 즉각적인 수정 조치가 필...,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28,Block reason:Identification of packaging defec...,블럭 원인:출하 전에 제품의 무결성을 손상시키는 포장 결함을 식별하여 즉각적인 수정...
2,3,368814,NORDVIKEN,"Bar table, 140x80 cm","바 테이블, 140x80 cm",P01,Bar furniture,바 가구,48663,107,48556,2024-03-01,Identification of packaging defects that compr...,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 수정 조치가 필요...,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28,Block reason:Identification of packaging defec...,블럭 원인:출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 수정 ...
3,4,368814,NORDVIKEN,"Bar table, 140x80 cm","바 테이블, 140x80 cm",P01,Bar furniture,바 가구,48663,107,48556,2024-03-01,Identification of packaging defects that compr...,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 수정 조치가 필요...,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28,Block reason:Identification of packaging defec...,블럭 원인:출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 수정 ...
4,5,368814,NORDVIKEN,"Bar table, 140x80 cm","바 테이블, 140x80 cm",P01,Bar furniture,바 가구,48663,107,48556,2024-03-01,Identification of packaging defects that compr...,출하 전에 제품의 무결성을 저해하는 포장 결함을 식별함으로써 즉각적인 수정 조치가 ...,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-04-28,Block reason:Identification of packaging defec...,블럭 원인:출하 전에 제품의 무결성을 저해하는 포장 결함을 식별함으로써 즉각적인 수...


In [21]:
# hdf = cc.sql('''DELETE FROM DEMO_BLOCK_3''')
# df_block = hdf.collect()
# df_block.head()

'NoneType' object is not iterable
Failed to retrieve data for the current dataframe, (0, 'No result set')


ProgrammingError: (0, 'No result set')

### Embedding - EN

#### Add Vector Column

In [42]:
# add a vector column to your table
cursor = cc.connection.cursor()
sql_command = '''ALTER TABLE DEMO_BLOCK_3 ADD (VECTOR_EN REAL_VECTOR(1536))'''
cursor.execute(sql_command)
cursor.close()

#### Get Embedding

In [3]:
# get embedding
from gen_ai_hub.proxy.native.openai import embeddings

def get_embedding(input, model="dc872f9eef04c31a") -> str:
    response = embeddings.create(
      deployment_id=model,
      input=input
    )
    return response.data[0].embedding

In [ ]:
#Test 
get_embedding("Nice to Meet you!")

NameError: name 'get_embedding' is not defined

In [52]:
# generate embeddings from the text
rows = []
for index, row in data2.iterrows():
    text = row['Context']
    # print(text)
    try:
        text_vector = get_embedding(input=text,model="dc872f9eef04c31a")
        myrow = (str(text_vector), row['BlockID'])
        rows.append(myrow)
    except Exception as e:
        print(e)

##### Import embeddings of Context to HANA

In [53]:
# bulk update
# cc.connection.setautocommit(False)
cursor = cc.connection.cursor()
sql = 'UPDATE DEMO_BLOCK_3 SET VECTOR_EN = TO_REAL_VECTOR(?) WHERE "BlockID" = ?'
try:
    cursor.executemany(sql, rows)
except Exception as e:
    cc.connection.rollback()
    print("An error occurred:", e)
try:
    cc.connection.commit()
finally:
    cursor.close()
cc.connection.setautocommit(True)

### Embedding - KR

In [54]:
# add a vector column to your table
cursor = cc.connection.cursor()
sql_command = '''ALTER TABLE DEMO_BLOCK_3 ADD (VECTOR_KR REAL_VECTOR(1536))'''
cursor.execute(sql_command)
cursor.close()

In [149]:
get_embedding(data2['Context_KR'][0])

[-0.020891809836030006,
 -0.015161988325417042,
 -0.0019448334351181984,
 -0.01342834997922182,
 -0.028840098530054092,
 0.030941033735871315,
 -0.03643578663468361,
 0.0007635169895365834,
 -0.00947624258697033,
 -0.0039190505631268024,
 -0.03869833052158356,
 0.008638806641101837,
 -0.0005734412115998566,
 -0.00267207995057106,
 -0.017497992143034935,
 0.04486890509724617,
 0.014420049265027046,
 -0.0054984246380627155,
 0.01371484063565731,
 -0.03711161017417908,
 -0.01877618208527565,
 0.01629326120018959,
 -8.29286000225693e-05,
 0.01201793272048235,
 -0.0022166327107697725,
 0.013795645907521248,
 0.03182254359126091,
 -0.02083304151892662,
 -0.01676340028643608,
 0.024050556123256683,
 0.0075001888908445835,
 0.006104463245719671,
 -0.012708448804914951,
 -0.011650635860860348,
 0.01032837014645338,
 0.007691182661801577,
 -0.001222178223542869,
 -0.017747754231095314,
 0.016116958111524582,
 0.0012947191717103124,
 0.012113429605960846,
 -0.006967609282582998,
 0.02319842949509

In [58]:
data2.shape

(8020, 19)

In [59]:
data2['Context_KR'][0]

'블럭 원인:출하 전에 제품의 무결성을 저해하는 포장 결함을 식별하여 즉각적인 시정 조치가 필요합니다.\n 지제 번호:368814\n 자재명:NORDVIKEN\n 자재 설명:바 테이블, 140x80 cm\n 자재 그룹:P01\n 구매 그룹 설명:바 가구'

In [60]:
print(data2.shape)
print(data2.drop_duplicates().shape)

(8020, 19)
(8020, 19)


In [61]:
# generate embeddings from the text
rows = []
for index, row in data2.iterrows():
    text = row['Context_KR']
    # print(text)
    try:
        text_vector = get_embedding(input=text,model="dc872f9eef04c31a")
        # text_vector = '[0, 1]'
        myrow = (str(text_vector), row['BlockID'])
        rows.append(myrow)
    except Exception as e:
        print(e)

Error code: 400 - {'error': {'code': None, 'message': "'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'param': None, 'type': 'invalid_request_error'}}
Error code: 400 - {'error': {'code': None, 'message': "'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'param': None, 'type': 'invalid_request_error'}}
Error code: 400 - {'error': {'code': None, 'message': "'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'param': None, 'type': 'invalid_request_error'}}
Error code: 400 - {'error': {'code': None, 'message': "'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'param': None, 'type': 'invalid_request_error'}}
Error code: 400 - {'error': {'code': None, 'message': "'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'param': 

In [62]:
# bulk update
# cc.connection.setautocommit(False)
cursor = cc.connection.cursor()
sql = 'UPDATE DEMO_BLOCK_3 SET VECTOR_KR = TO_REAL_VECTOR(?) WHERE "BlockID" = ?'
try:
    cursor.executemany(sql, rows)
except Exception as e:
    cc.connection.rollback()
    print("An error occurred:", e)
try:
    cc.connection.commit()
finally:
    cursor.close()
cc.connection.setautocommit(True)

In [63]:
# Check Updates
cc = ConnectionContext(address=url_c, port=port_c, user= user_c, password = pwd_c, encrypt=True)
cursor = cc.connection.cursor()
cursor.execute("""SET SCHEMA GEN_AI""")

hdf = cc.sql('''SELECT TOP 5 "BlockID","MaterialNumber", "MaterialName",
             "Blocked_stock","Block_reason","Blocked_date", "Solution", "Solution_KR","Solution_date", "VECTOR_KR"
             FROM DEMO_BLOCK_3''')
df_abstract = hdf.collect() 
df_abstract.head(5)

,BlockID,MaterialNumber,MaterialName,Blocked_stock,Block_reason,Blocked_date,Solution,Solution_KR,Solution_date,VECTOR_KR
0,5692,19011777,MELLTORP,38529,Inventory discrepancy detected during stock au...,2024-05-12,"[{""Instruction_step"": 1, ""Instruction"": ""Inves...","[{""Instruction_step"": 1, ""Instruction_KR"": ""감사...",2024-08-22,"[0, 6, 0, 0, 113, 133, 28, 188, 196, 138, 14, ..."
1,5692,19011777,MELLTORP,38529,Inventory discrepancy detected during stock au...,2024-05-12,"[{""Instruction_step"": 1, ""Instruction"": ""Inves...","[{""Instruction_step"": 1, ""Instruction_KR"": ""감사...",2024-08-22,"[0, 6, 0, 0, 113, 133, 28, 188, 196, 138, 14, ..."
2,5696,60214159,MALM,13145,Identification of packaging defects that compr...,2024-05-15,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-06-15,"[0, 6, 0, 0, 241, 121, 37, 187, 179, 74, 92, 1..."
3,5696,60214159,MALM,13145,Identification of packaging defects that compr...,2024-05-15,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-06-15,"[0, 6, 0, 0, 241, 121, 37, 187, 179, 74, 92, 1..."
4,5697,60214159,MALM,13145,Identification of packaging defects that compr...,2024-05-15,"[{""Instruction_step"": 1, ""Instruction"": ""Condu...","[{""Instruction_step"": 1, ""Instruction_KR"": ""현재...",2024-06-15,"[0, 6, 0, 0, 183, 204, 88, 187, 64, 185, 43, 1..."


## Create an Embedding to query the HANA Table 

In [55]:
text_vector = get_embedding(input = "의자의 포장 불량 배송에 대한 해결책은 무엇입니까?")

In [58]:
len(text_vector)

1536

In [64]:
with open('/Users/i551982/Desktop/Github/enablement-session-01/vector_demo2/text_vector_01_KR.txt','w') as output: 
    output.write(str(text_vector))

In [65]:
text_vector = get_embedding(input = "What can be solutions for the delivery for packing defects for electonics materials?")

with open('/Users/i551982/Desktop/Github/enablement-session-01/vector_demo2/text_vector_01_EN.txt','w') as output: 
    output.write(str(text_vector))